In [1]:
# !pip3 install textgrid
import textgrid

In [2]:
import os

os.environ['AWS_ENDPOINT_URL'] = 'https://s3.us-west-000.backblazeb2.com'
os.environ['AWS_ACCESS_KEY_ID'] = ''
os.environ['AWS_SECRET_ACCESS_KEY'] = ''

In [3]:
from tqdm import tqdm
import boto3
import subprocess
from unidecode import unidecode
import IPython.display as ipd
from glob import glob
import soundfile as sf

os.makedirs('part5-same', exist_ok = True)
os.makedirs('part5-same-audio', exist_ok = True)
os.makedirs('part5-separate', exist_ok = True)
os.makedirs('part5-separate-audio', exist_ok = True)

In [4]:
!mkdir part5-same-audio-mp3
!mkdir part5-separate-audio-mp3
!rm part5-same-audio-mp3/*.mp3
!rm part5-separate-audio-mp3/*.mp3

mkdir: cannot create directory ‘part5-same-audio-mp3’: File exists
mkdir: cannot create directory ‘part5-separate-audio-mp3’: File exists
rm: cannot remove 'part5-same-audio-mp3/*.mp3': No such file or directory
rm: cannot remove 'part5-separate-audio-mp3/*.mp3': No such file or directory


In [5]:
session = boto3.Session()

In [6]:
s3 = session.resource('s3')
bucket = s3.Bucket('imda-singapore')

In [7]:
for f in tqdm(bucket.objects.filter(Prefix='IMDA - National Speech Corpus (Additional)/PART5/Debate Scripts')):
    filename = os.path.join('part5-same', os.path.split(f.key)[1])
    if os.path.exists(filename):
        continue
                            
    bucket.download_file(f.key, filename)

1500it [00:01, 1248.53it/s]


In [8]:
for f in tqdm(bucket.objects.filter(Prefix='IMDA - National Speech Corpus (Additional)/PART5/Debate Audio')):
    filename = os.path.join('part5-same-audio', os.path.split(f.key)[1])
    if os.path.exists(filename):
        continue
                            
    bucket.download_file(f.key, filename)

1500it [26:30,  1.06s/it]


In [9]:
for f in tqdm(bucket.objects.filter(Prefix='IMDA - National Speech Corpus (Additional)/PART5/Finance + Emotion Scripts')):
    filename = os.path.join('part5-separate', os.path.split(f.key)[1])
    if os.path.exists(filename):
        continue
                            
    bucket.download_file(f.key, filename)

4500it [1:11:03,  1.06it/s]


In [14]:
for f in tqdm(bucket.objects.filter(Prefix='IMDA - National Speech Corpus (Additional)/PART5/Finance + Emotions Audio')):
    filename = os.path.join('part5-separate-audio', os.path.split(f.key)[1])
    if os.path.exists(filename):
        continue
                            
    bucket.download_file(f.key, filename)

4503it [1:24:53,  1.13s/it]


In [16]:
import re

def cleaning(s):
    return re.sub(r'[ ]+', ' ', s).strip()

replaces = [
    '<malay>',
    '</malay>',
    '_',
    '<UNK>',
    '(ppl)',
    '(ppo)',
    '(mm)',
    '(um)',
    '(ppb)'
]

replaces_with = {
    '[lah]': 'lah',
    '[ah]': 'ah',
    '[sia]': 'sia',
    '[eh]': 'eh',
    '(uh)': 'uh',
    '[what]': 'what',
    '[oh]': 'oh',
    '(err)': 'err',
    '[lor]': 'lor',
    '[ha]': 'ha',
    '[meh]': 'meh',
    '[one]': 'one',
    "[a'ah]": "a'ah",
    '[hor]': 'hor',
    '[leh]': 'leh',
    '[mah]': 'mah',
    '[nah]': 'nah',
    '[tau]': 'tau',
    '[uh]': 'uh',
    '[wah]': 'wah'
}

maxlen = 15

In [21]:
scripts = sorted(glob('part5-same/*'))

all_results = {}
for s in tqdm(scripts):
    results = []
    id_file = os.path.split(s)[1].replace('.TextGrid', '')
    texts = []
    total_l = 0
    start = []
    end = []
    try:
        tg = textgrid.TextGrid.fromFile(s)
    except:
        continue
    for tg_ in tg[0]:
        if tg_.mark is None:
            continue
        if not len(tg_.mark):
            continue
        start.append(tg_.minTime)
        end.append(tg_.maxTime)
        total_l += (tg_.maxTime - tg_.minTime)
        if tg_.mark[0] != '<' and tg_.mark[-1] != '>':
            t = tg_.mark.split()
            filtered = []
            for no, t_ in enumerate(t):
                if t_[-1] == '~':
                    continue
                if t_[0] == '#' and t_[-1] == '#':
                    t_ = t_[1:-1].title()
                filtered.append(t_)
                t[no] = filtered

            texts.append(' '.join(filtered))
        if total_l >= maxlen:
            t = ', '.join(texts)
            if len(t) > 3:
                for r in replaces:
                    t = t.replace(r, '')
                for r, v in replaces_with.items():
                    t = t.replace(r, v)
                t = cleaning(t)
                results.append((t, [start[0], end[-1]]))
            texts = []
            total_l = 0
            start = []
            end = []
    
    if len(texts):
        t = ', '.join(texts)
        if len(t) > 3:
            for r in replaces:
                t = t.replace(r, '')
            for r, v in replaces_with.items():
                t = t.replace(r, v)
            t = cleaning(t)
            results.append((t, [start[0], end[-1]]))
    
    all_results[id_file] = results

100%|██████████| 1500/1500 [00:05<00:00, 276.66it/s]


In [22]:
keys = list(all_results)
keys = [(k, all_results[k]) for k in keys]

In [23]:
keys[0]

('app_4001_6001_phnd_deb-1',
 [('art Ming-Guan, okay so uh I guess it’s time for us to debate the topic uh technology creates more problem than benefits for the society so err err have you uh done any uh research on that topic',
   [0.0, 28.18878]),
  ('so ya the proponent opposition, oh you want to you want some time to do the do the this your your do you do you need sometime to do the planning cause I I okay I basically I just wrote down some some of the points on my in the if me lah still got some points lah but I think you should start first so ya I wait for you',
   [28.18878, 48.97801]),
  ('okay sure', [48.97801, 73.1]),
  ('okay ya really so I think you are the propositions',
   [98.41488, 118.47419]),
  ('okay', [118.47419, 147.38]),
  ('okay Ming-Guan so are you done with your err proposition statement, okay so uh my turn so basically I am the opposition so basically I suppose to argue that technology err create more benefits than problems for the society',
   [255.97814, 278

In [25]:
def loop(keys):
    results = []
    keys, _ = keys
    for key in tqdm(keys):
        key, rows = key
        audio_file = os.path.join('part5-same-audio', f'{key}.wav')
        if not os.path.exists(audio_file):
            continue
        y, sr = sf.read(audio_file)
        index = 0
        for row in rows:
            filename = os.path.join('part5-same-audio-mp3', f'{key}-{index}.mp3')
            if os.path.exists(filename):
                index += 1
                
                results.append({
                    'filename': filename,
                    'text': row[0]
                })
                
                continue
                
            y_ = y[int(sr * row[-1][0]): int(sr * row[-1][-1])]
            sf.write(filename, y_, sr)
            index += 1
            results.append({
                'filename': filename,
                'text': row[0]
            })
    return results

In [30]:
import mp

results = mp.multiprocessing(keys, loop, cores = 20)

100%|██████████| 74/74 [12:02<00:00,  9.76s/it]


In [31]:
len(results)

51982

In [32]:
results[-2]

{'filename': 'part5-same-audio-mp3/app_4258_6516_phnd_deb-3-19.mp3',
 'text': ", that there's actually a kind of importance to to a book even though it's kind of , uh it might have lose err it it it might have lose kind of like"}

In [33]:
ipd.Audio(results[-2]['filename'], rate = 16000)

In [34]:
import json

with open('imda-same-part5.jsonl', 'w') as fopen:
    for r in tqdm(results):
        fopen.write(f'{json.dumps(r)}\n')

100%|██████████| 51982/51982 [00:00<00:00, 278995.17it/s]


In [35]:
!du -hs part5-same-audio-mp3

5.6G	part5-same-audio-mp3


In [36]:
scripts = sorted(glob('part5-separate/*'))

all_results = {}
for s in tqdm(scripts):
    results = []
    id_file = os.path.split(s)[1].replace('.TextGrid', '')
    texts = []
    total_l = 0
    start = []
    end = []
    try:
        tg = textgrid.TextGrid.fromFile(s)
    except:
        continue
    for tg_ in tg[0]:
        if tg_.mark is None:
            continue
        if not len(tg_.mark):
            continue
        start.append(tg_.minTime)
        end.append(tg_.maxTime)
        total_l += (tg_.maxTime - tg_.minTime)
        if tg_.mark[0] != '<' and tg_.mark[-1] != '>':
            t = tg_.mark.split()
            filtered = []
            for no, t_ in enumerate(t):
                if t_[-1] == '~':
                    continue
                if t_[0] == '#' and t_[-1] == '#':
                    t_ = t_[1:-1].title()
                filtered.append(t_)
                t[no] = filtered

            texts.append(' '.join(filtered))
        if total_l >= maxlen:
            t = ', '.join(texts)
            if len(t) > 3:
                for r in replaces:
                    t = t.replace(r, '')
                for r, v in replaces_with.items():
                    t = t.replace(r, v)
                t = cleaning(t)
                results.append((t, [start[0], end[-1]]))
            texts = []
            total_l = 0
            start = []
            end = []
    
    if len(texts):
        t = ', '.join(texts)
        if len(t) > 3:
            for r in replaces:
                t = t.replace(r, '')
            for r, v in replaces_with.items():
                t = t.replace(r, v)
            t = cleaning(t)
            results.append((', '.join(texts), [start[0], end[-1]]))
    
    all_results[id_file] = results

100%|██████████| 4500/4500 [00:20<00:00, 224.29it/s]


In [37]:
keys = list(all_results)
keys = [(k, all_results[k]) for k in keys]

In [38]:
def loop(keys):
    results = []
    keys, _ = keys
    for key in tqdm(keys):
        key, rows = key
        audio_file = os.path.join('part5-separate-audio', f'{key}.wav')
        if not os.path.exists(audio_file):
            continue
        y, sr = sf.read(audio_file)
        index = 0
        for row in rows:
            filename = os.path.join('part5-separate-audio-mp3', f'{key}-{index}.mp3')
            if os.path.exists(filename):
                index += 1
                
                results.append({
                    'filename': filename,
                    'text': row[0]
                })
                
                continue
                
            y_ = y[int(sr * row[-1][0]): int(sr * row[-1][-1])]
            sf.write(filename, y_, sr)
            index += 1
            results.append({
                'filename': filename,
                'text': row[0]
            })
    return results

In [ ]:
import mp

results_separate = mp.multiprocessing(keys, loop, cores = 20)

 26%|██▌       | 58/224 [12:02<36:09, 13.07s/it]

In [41]:
len(results_separate)

215412

In [42]:
results_separate[1000]

{'filename': 'part5-separate-audio-mp3/app_3004_4007_phnd_pos-1.mp3',
 'text': "the first time I've tasted such good indian food and it was so delicious that I was telling him like this the best indian food I've eaten in my life ya so that's actually one of like the one of the best meals I've had"}

In [43]:
ipd.Audio(results_separate[1000]['filename'], rate = 16000)

In [44]:
with open('imda-separate-part5.jsonl', 'w') as fopen:
    for r in tqdm(results_separate):
        fopen.write(f'{json.dumps(r)}\n')

100%|██████████| 215412/215412 [00:00<00:00, 278717.22it/s]


In [45]:
from huggingface_hub import HfApi
api = HfApi()

In [46]:
api.upload_file(
    path_or_fileobj='imda-same-part5.jsonl',
    path_in_repo='imda-same-part5.jsonl',
    repo_id='mesolitica/IMDA-STT',
    repo_type='dataset',
)

imda-same-part5.jsonl:   0%|          | 0.00/15.3M [00:00<?, ?B/s]

'https://huggingface.co/datasets/mesolitica/IMDA-STT/blob/main/imda-same-part5.jsonl'

In [47]:
api.upload_file(
    path_or_fileobj='imda-separate-part5.jsonl',
    path_in_repo='imda-separate-part5.jsonl',
    repo_id='mesolitica/IMDA-STT',
    repo_type='dataset',
)

imda-separate-part5.jsonl:   0%|          | 0.00/55.3M [00:00<?, ?B/s]

'https://huggingface.co/datasets/mesolitica/IMDA-STT/blob/main/imda-separate-part5.jsonl'

In [51]:
# !sudo apt install p7zip-full p7zip-rar -y

In [52]:
!7z -v5g a part5-same-audio-mp3.7z part5-same-audio-mp3 > /dev/null
!7z -v5g a part5-separate-audio-mp3.7z part5-separate-audio-mp3 > /dev/null

In [53]:
for f in glob('part5-same-audio-mp3.7z*'):
    print(f)
    api.upload_file(
        path_or_fileobj=f,
        path_in_repo=f,
        repo_id='mesolitica/IMDA-STT',
        repo_type='dataset',
    )

part5-same-audio-mp3.7z.001


part5-same-audio-mp3.7z.001:   0%|          | 0.00/5.37G [00:00<?, ?B/s]

The history saving thread hit an unexpected error (OperationalError('database is locked')).History will not be written to the database.
part5-same-audio-mp3.7z.002


part5-same-audio-mp3.7z.002:   0%|          | 0.00/365M [00:00<?, ?B/s]

In [54]:
for f in glob('part5-separate-audio-mp3.7z*'):
    print(f)
    api.upload_file(
        path_or_fileobj=f,
        path_in_repo=f,
        repo_id='mesolitica/IMDA-STT',
        repo_type='dataset',
    )

part5-separate-audio-mp3.7z.002


part5-separate-audio-mp3.7z.002:   0%|          | 0.00/5.37G [00:00<?, ?B/s]

part5-separate-audio-mp3.7z.001


part5-separate-audio-mp3.7z.001:   0%|          | 0.00/5.37G [00:00<?, ?B/s]

part5-separate-audio-mp3.7z.005


part5-separate-audio-mp3.7z.005:   0%|          | 0.00/280M [00:00<?, ?B/s]

part5-separate-audio-mp3.7z.004


part5-separate-audio-mp3.7z.004:   0%|          | 0.00/5.37G [00:00<?, ?B/s]

part5-separate-audio-mp3.7z.003


part5-separate-audio-mp3.7z.003:   0%|          | 0.00/5.37G [00:00<?, ?B/s]

In [55]:
total_len = 0
for f in tqdm(glob('part5-separate-audio/*.wav')):
    y, sr = sf.read(f)
    total_len += len(y) / sr

100%|██████████| 4500/4500 [04:39<00:00, 16.10it/s]


In [56]:
for f in tqdm(glob('part5-same-audio/*.wav')):
    y, sr = sf.read(f)
    total_len += len(y) / sr

100%|██████████| 1500/1500 [01:46<00:00, 14.02it/s]


In [57]:
total_len / 60 / 60

2044.5220318402826

In [58]:
!rm -rf part5-same-audio part5-separate-audio

In [59]:
!rm *7z*